In [ ]:
covid_sentences = [
    "The COVID-19 vaccine rollout has been effective in reducing the number of cases.",
    "New variants of the coronavirus are causing a lot of concern among health experts.",
    "Many people are hesitant about the COVID vaccine due to misinformation.",
    "Hospitals are overwhelmed due to the surge in COVID-19 cases.",
    "The pandemic has led to a significant increase in mental health issues worldwide.",
    "Researchers are studying the long-term effects of COVID-19 on the human body.",
    "Remote work has become the new norm since the COVID pandemic started.",
    "Many countries are implementing stricter lockdowns to control the spread of COVID-19.",
]


In [ ]:
class LDA():

    def __init__(self, n_topics, n_iter, alpha, beta):
        
        pass

    